# $\mathcal{H}(2^4)$ Branching Occurs Over All 2-Torsion Points

This notebook addresses the case where each of the four $2$-torsion points has exactly one double zero above it.

We recall that in $\mathcal{H}(2^4)$, $d_{opt} = 9$.

In [1]:
import re

## Step 1

In [2]:
#This loads all of the 1-cylinder diagrams in (all connected components of the stratum) H(2,2) formatted as python lists

H2_2_cyl_diags = [[[[0, 5, 3, 1, 2, 4], [0, 5, 3, 1, 2, 4]]], 
                  [[[0, 5, 2, 1, 4, 3], [0, 4, 2, 1, 5, 3]]], 
                  [[[0, 5, 2, 3, 1, 4], [0, 3, 1, 5, 2, 4]]], 
                  [[[0, 5, 3, 2, 1, 4], [0, 1, 3, 4, 5, 2]]]]

#This loads all of the functions for processing cylinder diagrams

#Only the general cylinder diagram functions are needed in this case
%run ./ST5_fcns/cyl_diag_fcns.ipynb

#In the previous case finding a simple zero sufficed to extract potential odd length saddle connections
#In this case it's the switch from one zero to another that marks an odd saddle connection

def extract_switch(vertex_labels):
    cyl_bot = vertex_labels[0][0]
    cyl_bot_int = [int(j[1:]) for j in cyl_bot]
#    print(cyl_bot_int)
    length = len(cyl_bot)
    switch_list = []
    for i in enumerate(cyl_bot_int):
        if i[1] == cyl_bot_int[(i[0]+1)%length]:
            switch_list += [0]
        else:
            switch_list += [1]
    return switch_list

def strat_odd_sc_2t4(master_list):
    #This is the main function to run to get the processed data
    #The data consists of 
    total = []
    for i in master_list:
        print(i, ' from master')
        print(vertex_labels(i))
        total.append((i,[extract_switch(vertex_labels(i))]))
    return total

In [3]:
#This runs instantly and constructs all of the binary lists

H2_2_vertex_data = strat_odd_sc_2t4(H2_2_cyl_diags)

([[[0, 5, 3, 1, 2, 4], [0, 5, 3, 1, 2, 4]]], ' from master')
[[['V0', 'V1', 'V0', 'V1', 'V0', 'V1'], ['V0', 'V1', 'V0', 'V1', 'V0', 'V1']]]
([[[0, 5, 2, 1, 4, 3], [0, 4, 2, 1, 5, 3]]], ' from master')
[[['V0', 'V0', 'V1', 'V1', 'V1', 'V0'], ['V0', 'V1', 'V1', 'V1', 'V0', 'V0']]]
([[[0, 5, 2, 3, 1, 4], [0, 3, 1, 5, 2, 4]]], ' from master')
[[['V0', 'V1', 'V0', 'V1', 'V0', 'V1'], ['V0', 'V1', 'V0', 'V1', 'V0', 'V1']]]
([[[0, 5, 3, 2, 1, 4], [0, 1, 3, 4, 5, 2]]], ' from master')
[[['V0', 'V1', 'V0', 'V1', 'V1', 'V0'], ['V0', 'V1', 'V0', 'V0', 'V1', 'V1']]]


Among partitions of $18$ into $6$ numbers the minimum of the maximum numbers among all partitions is $2d_{opt}/6 = 3$, which is $3$ in this case.

Solving $18 - 2(3) - 2t_0 \geq 0$ implies that the largest value of $t_0$ is $6$.

Since the situation is symmetric, the same bound applies to $s_0$.

In summary:

$$3 \leq s_0, t_0 \leq 6$$

## Step 2

Unlike the cases where $d_{opt}$ was even, this case has very few restrictions on the partitions.  There must be at least two odd numbers in the partitions, but any positive even number is permissible.  Therefore, we rewrite the usual partition functions without the usual restrictions.

In [4]:
def partition_pre(n, d, t_max = None, depth=0):
    #Function pulled from https://stackoverflow.com/questions/10035752/elegant-python-code-for-integer-partitioning
    #Written by Nico Schlömer
    if t_max == None:
        t_max = n+1
    if d == depth:
        return [[]]
    return [
        item + [i]
        for i in range(1,min(n+1,t_max+1))
        for item in partition_pre(n-i, d, t_max = t_max, depth=depth+1)
        ]

def t_max_check(part, t_max):
    for j in part:
        if j > t_max:
            return False
    return True

def partition(n, d, t_max):
    #Function pulled from https://stackoverflow.com/questions/10035752/elegant-python-code-for-integer-partitioning
    #Written by Nico Schlömer
    #Key in the if statement below is that the functions are sorted in terms of computational complexity
    #The fastest check is performed first and the most intensive last
    return [[n-sum(p)] + p for p in partition_pre(n-1, d-1, t_max = t_max) 
            if t_max_check([n-sum(p)] + p, t_max)]

def odd_number_count_check(part):
    if sum([j%2 for j in part]) >= 2:
        return True
    else:
        return False

def create_sc_partition_file_big_tuple(t_tuple_begin, d_opt, part_length, filename_root):
    width = 2*d_opt
    t_list_begin = list(t_tuple_begin)
    n = width - sum(t_list_begin)
    d = part_length-len(t_list_begin)
    end_part_list = partition(n, d, t_max = t_list_begin[0])
    part_list = [tuple(t_list_begin + part) for part in end_part_list 
                 if odd_number_count_check(t_list_begin + part)]
    file_tag = ''
    for t in t_list_begin:
        file_tag += str(t) + '_'
    file_tag = file_tag[:-1]
    filename = filename_root + file_tag
    with open(filename, 'w') as file:
        file.write(str(part_list))
    print(filename + ' written')
    return 'done'

def create_sc_partition_file(t_tuple_begin, d_opt, part_length, filename_root, t0_range):
    width = 2*d_opt
    try:
        t_tuple_begin_list = list(t_tuple_begin)
    except:
        t_tuple_begin_list = [int(t_tuple_begin)]
    if len(t_tuple_begin_list) > 0:
        return create_sc_partition_file_big_tuple(t_tuple_begin_list, width, part_length, filename_root)
    else:
        total = []
        for t0 in t0_range:
            t_list_begin = [t0]
            n = width - t0
            d = part_length - 1
            end_part_list = partition(n, d, t_max = t_list_begin[0])
            part_list = [tuple(t_list_begin + part) for part in end_part_list 
                         if odd_number_count_check(t_list_begin + part)]
            total += part_list
        with open(filename_root, 'w') as file:
            file.write(str(total))
        print(filename_root + ' written')
        return 'done'

In [5]:
if True:
    create_sc_partition_file((), part_length = 6, t0_range = range(3,7), d_opt = 9,
                             filename_root = 'ST5_data//H_2t4//4_branch_point//partitions//H2_2_part')

#Load the partitions

if True:
    with open('ST5_data//H_2t4//4_branch_point//partitions//H2_2_part', 'r') as file:
        H2_2_part = eval(file.read())

ST5_data//H_2t4//4_branch_point//partitions//H2_2_part written


## Step 3

In [6]:
#This loads all of the align_list functions needed for nearly every case
#This includes the align_list evaluate function

%run ./ST5_fcns/align_list_fcns.ipynb

if True:
    for t in range(3,7):
        align_list_write_file(H2_2_part, H2_2_vertex_data, 
                              'ST5_data//H_2t4//4_branch_point//align_list//H_2_2_align_list', 
                              t0 = t)

1 partitions to search
125 partitions to search
320 partitions to search
300 partitions to search


## Step 4

In [7]:
#This loads all of the visible_align_list functions needed for nearly every case

%run ./ST5_fcns/align_list_visible_fcns.ipynb

if True:
    generate_all_align_list_visible_files(range(3,7), range(3,7), 9, 
                                          'ST5_data//H_2t4//4_branch_point//', 
                                          'align_list//H_2_2_align_list_', 
                                          'align_list_visible//H_2_2_align_list_')

total_list generated
ST5_data//H_2t4//4_branch_point//align_list//H_2_2_align_list_3 read
2 configurations to search
ST5_data//H_2t4//4_branch_point//align_list_visible//H_2_2_align_list_3_visible_3 written
ST5_data//H_2t4//4_branch_point//align_list//H_2_2_align_list_3 read
2 configurations to search
ST5_data//H_2t4//4_branch_point//align_list_visible//H_2_2_align_list_3_visible_4 written
ST5_data//H_2t4//4_branch_point//align_list//H_2_2_align_list_3 read
2 configurations to search
ST5_data//H_2t4//4_branch_point//align_list_visible//H_2_2_align_list_3_visible_5 written
ST5_data//H_2t4//4_branch_point//align_list//H_2_2_align_list_3 read
2 configurations to search
ST5_data//H_2t4//4_branch_point//align_list_visible//H_2_2_align_list_3_visible_6 written
ST5_data//H_2t4//4_branch_point//align_list//H_2_2_align_list_4 read
30 configurations to search
ST5_data//H_2t4//4_branch_point//align_list_visible//H_2_2_align_list_4_visible_3 written
ST5_data//H_2t4//4_branch_point//align_list//H_2

## Step 5

In [8]:
#At this point, none of the files above need to be loaded for this to run.
#Once the files above are generated, the file sizes are small enough that these can be loaded on the fly.

#This loads all of the visible_align_list functions needed for nearly every case

%run ./ST5_fcns/combine_align_list_visible_fcns.ipynb

if True:
    combine_align_list_visible_write_file(s_range = range(3,7), 
                                          s_filename_root = 'align_list_visible//H_2_2_align_list_', 
                                          t_range = range(3,7), 
                                          t_filename_root = 'align_list_visible//H_2_2_align_list_',
                                          d_opt = 9, 
                                          root_dir = 'ST5_data//H_2t4//4_branch_point//')

ST5_data//H_2t4//4_branch_point//align_list_visible//H_2_2_align_list_3_visible_3 read
ST5_data//H_2t4//4_branch_point//align_list_visible//H_2_2_align_list_3_visible_3 read
6*6=36 combinations to check
ST5_data//H_2t4//4_branch_point//align_list_visible//H_2_2_align_list_4_visible_3 read
ST5_data//H_2t4//4_branch_point//align_list_visible//H_2_2_align_list_3_visible_4 read
empty list
ST5_data//H_2t4//4_branch_point//align_list_visible//H_2_2_align_list_5_visible_3 read
ST5_data//H_2t4//4_branch_point//align_list_visible//H_2_2_align_list_3_visible_5 read
282*6=1692 combinations to check
ST5_data//H_2t4//4_branch_point//align_list_visible//H_2_2_align_list_6_visible_3 read
ST5_data//H_2t4//4_branch_point//align_list_visible//H_2_2_align_list_3_visible_6 read
2*6=12 combinations to check
ST5_data//H_2t4//4_branch_point//align_list_visible//H_2_2_align_list_3_visible_4 read
ST5_data//H_2t4//4_branch_point//align_list_visible//H_2_2_align_list_4_visible_3 read
empty list
ST5_data//H_2t4//

## Step 6

In [1]:
#This loads all of the vertical permutation check functions needed for nearly every case
#This function can be run once admissible_list is written and without needing to load any other file

%run ./ST5_fcns/vert_perm_check_fcns.ipynb

In [2]:
#This checks that all (both) of the vertical permutations have the correct length of 2*d_opt

if True:
    vert_perm_check_file(9, 'ST5_data//H_2t4//4_branch_point//')

admissible_list read with 6792 candidates
admissible_list_vert_perm_check written with 2520 elements


In [3]:
if True:
    all_vert_perm_check_file(9, 'ST5_data//H_2t4//4_branch_point//')

admissible_list_vert_perm_check read with 2520 candidates
admissible_list_all_vert_perm_check written with 2088 elements


## Step 7

In [5]:
%run ./ST5_fcns/slope_test_fcn.ipynb

if True:
    final_list = slope_test(9, 'ST5_data//H_2t4//4_branch_point//')

admissible_list_all_vert_perm_check read with 2088 candidates
vert_perm_total_1 produced with 900 elements
vert_perm_total_2 produced with 0 elements
All possibilities have been excluded
